# Seq2Seq Model for Joint Angle Prediction and Uncertainty Quantification

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import math, random, time, os

import torch
from torch import nn, optim, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker

from common.data_utils import *
from common.models import *
from common.training_utils import *

torch.manual_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset Management

In [4]:
DATA_PATH = Path('../csv-files')
FILENAMES = ['IMU_S1_2017_04_02_1.csv','IMU_S2_2017_04_10_1a_a1.csv',
             'IMU_S1_2017_04_11_03a_a_1.csv']

joint_requests = {'Joint' : ['jL5S1','jL4L3','jL1T12','jT9T8','jT1C7','jC1Head',
                             'jRightC7Shoulder','jRightShoulder','jRightElbow','jRightWrist',
                             'jLeftC7Shoulder','jLeftShoulder','jLeftElbow','jLeftWrist',
                             'jRightHip','jRightKnee','jRightAnkle','jRightBallFoot',
                             'jLeftHip','jLeftKnee','jLeftAnkle','jLeftBallFoot']}

In [5]:
positions = None
joint_angles = None
seq_length = 120
files = ['IMU_S1_2017_04_02_1.csv','IMU_S2_2017_04_10_1a_a1.csv',
         'IMU_S1_2017_04_11_03a_a_1.csv']

for idx, file in enumerate(files):
    
    joint_target_columns = request_indices(joint_requests)
    
    print("Index: " + str(idx + 1), end='\r')
    
    joint_angles_temp = np.loadtxt(DATA_PATH / file, delimiter=',', usecols=joint_target_columns)
    
    discard_remainder(joint_angles_temp, seq_length)
    
    joint_angles_temp *= (np.pi/180.)
    
    if idx == 0:
        joint_angles = joint_angles_temp
    else:
        joint_angles = np.concatenate((joint_angles, joint_angles_temp), axis=0)
print("Done with reading files")

joint_angles = preprocessing.normalize(joint_angles, axis=0)

joint_angles = reshape_to_sequences(joint_angles, seq_length=seq_length) 

data = joint_angles

Done with reading files


In [6]:
encoder_input_data, decoder_target_data = split_sequences(data, seq_length=seq_length)

batch_size = 32
encoder_input_data = discard_remainder(encoder_input_data, batch_size)
decoder_target_data = discard_remainder(decoder_target_data, batch_size)

## Refactor using Dataset
------------------------------

PyTorch has an abstract Dataset class.  A Dataset can be anything that has
a ``__len__`` function (called by Python's standard ``len`` function) and
a ``__getitem__`` function as a way of indexing into it.
__[This tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)__
walks through a nice example of creating a custom ``FacialLandmarkDataset`` class
as a subclass of ``Dataset``.

PyTorch's __[TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset)__
is a Dataset wrapping tensors. **By defining a length and way of indexing,
this also gives us a way to iterate, index, and slice along the first
dimension of a tensor.** This will make it easier to access both the
independent and dependent variables in the same line as we train.


In [7]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [8]:
encoder_input_data = torch.tensor(encoder_input_data).to(device)
decoder_target_data = torch.tensor(decoder_target_data).to(device)

dataset = TensorDataset(encoder_input_data, decoder_target_data)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Seq2Seq Model

### Variational Dropout

I used variational dropout that I found __[here](https://discuss.pytorch.org/t/variational-dropout/23030/4)__.

In [9]:
class VariationalDropout(nn.Module):
    def __init__(self, alpha=1.0, dim=None):
        super(VariationalDropout, self).__init__()
        
        self.dim = dim
        self.max_alpha = alpha
        # Initial alpha
        log_alpha = (torch.ones(dim) * alpha).log()
        self.log_alpha = nn.Parameter(log_alpha)
        
    def kl(self):
        c1 = 1.16145124
        c2 = -1.50204118
        c3 = 0.58629921
        
        alpha = self.log_alpha.exp()
        
        negative_kl = 0.5 * self.log_alpha + c1 * alpha + c2 * alpha**2 + c3 * alpha**3
        
        kl = -negative_kl
        
        return kl.mean()
    
    def forward(self, x):
        """
        Sample noise   e ~ N(1, alpha)
        Multiply noise h = h_ * e
        """
        if self.train():
            # N(0,1)
            epsilon = Tensor(torch.randn(x.size()))
            if x.is_cuda:
                epsilon = epsilon.cuda()

            # Clip alpha
            self.log_alpha.data = torch.clamp(self.log_alpha.data, max=self.max_alpha)
            alpha = self.log_alpha.exp()

            # N(1, alpha)
            epsilon = epsilon * alpha

            return x * epsilon
        else:
            return x

### Encoder

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.gru = nn.GRU(input_size, hidden_size)
        self.dropout = VariationalDropout(alpha=0.05, dim=(1, batch_size, hidden_size))

    def forward(self, input, hidden):
        output, hidden = self.gru(input, hidden)
        output = self.dropout(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=device)

### Decoder

In [11]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.gru = nn.GRU(input_size, hidden_size)
        self.dropout = nn.Dropout(0.05)
        self.out = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
        
    def forward(self, input, hidden):
        output, hidden = self.gru(input, hidden)
        output = self.dropout(output)
        output = self.tanh(self.out(output))
        output = self.dropout(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_size, device=device)

## Training

In [13]:
def get_encoder(num_features, hidden_size=64, lr=0.001, bs=batch_size):
    encoder = EncoderRNN(num_features, hidden_size, bs).to(device)
    return encoder, optim.Adam(encoder.parameters(), lr=lr)

def get_decoder(num_features, hidden_size=64, lr=0.001, bs=batch_size):
    decoder = DecoderRNN(num_features, hidden_size, num_features, bs).to(device)
    return decoder, optim.Adam(decoder.parameters(), lr=lr)

In [14]:
def train(data, models, opts, criterion, teacher_forcing_ratio):
    loss = 0
    
    encoder, decoder = models
    
    if opts is not None:
        encoder_opt, decoder_opt = opts 
    
    input_batch, target_batch = data
    
    seq_length = input_batch.shape[1]

    encoder_hidden = encoder.initHidden()

    for t in range(seq_length):
        input = input_batch[:, t, :].unsqueeze(0).float()
        _, encoder_hidden = encoder(input, encoder_hidden)

    decoder_hidden = encoder_hidden
    decoder_input = torch.ones_like(target_batch[:, 0, :]).unsqueeze(0).float()
    EOS = torch.zeros_like(target_batch[:, 0, :]).unsqueeze(0).float()

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    for t in range(seq_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        target = target_batch[:, t, :].unsqueeze(0).float()
        loss += criterion(decoder_output, target)
        if torch.all(torch.eq(decoder_output, EOS)):
            break
        
        if use_teacher_forcing:
            decoder_input = target
        else:
            decoder_input = decoder_output.detach()

    if opts is not None:
        loss.backward()
        
        encoder_opt.step()
        encoder_opt.zero_grad()

        decoder_opt.step()
        decoder_opt.zero_grad()
    
    return loss.item() / seq_length

In [15]:
def fit(models, opts, epochs, dataloaders, criterion, teacher_forcing_ratio):
    
    train_dataloader, val_dataloader = dataloaders
    
    for epoch in range(epochs):

        for index, data in enumerate(train_dataloader, 0):
            loss = train(data, models, 
                         opts, criterion,
                         teacher_forcing_ratio) 
            print("Iteration: " + str(index + 1), "Loss: " + str(loss), end="\r")

        with torch.no_grad():
            # calculating loss for validation
            losses = [train(data, models, None, criterion, teacher_forcing_ratio) 
                      for _, data in enumerate(val_dataloader, 0)]
        val_loss = np.sum(losses) / len(losses)
        print()
        print("Epoch: " + str(epoch + 1), "Training Loss: " + str(loss), "Val Loss: " + str(val_loss))

In [16]:
epochs = 10

encoder, encoder_opt = get_encoder(66)
decoder, decoder_opt = get_decoder(66)
teacher_forcing_ratio = 0.0

models = (encoder, decoder)
opts = (encoder_opt, decoder_opt)

dataloaders = (train_dataloader, val_dataloader)

criterion = nn.L1Loss()
loss = 0

fit(models, opts, epochs, dataloaders, criterion, teacher_forcing_ratio)

Iteration: 176 Loss: 0.07834654649098714
Epoch: 1 Training Loss: 0.07834654649098714 Val Loss: 0.08326163996349682
Iteration: 176 Loss: 0.07959001064300537
Epoch: 2 Training Loss: 0.07959001064300537 Val Loss: 0.07075883402968898
Iteration: 176 Loss: 0.066588171323140464
Epoch: 3 Training Loss: 0.06658817132314046 Val Loss: 0.06556793812549476
Iteration: 176 Loss: 0.065823249022165945
Epoch: 4 Training Loss: 0.06582324902216594 Val Loss: 0.06284450292587282
Iteration: 176 Loss: 0.063518480459849045
Epoch: 5 Training Loss: 0.06351848045984904 Val Loss: 0.060856935562509484
Iteration: 176 Loss: 0.054184639453887944
Epoch: 6 Training Loss: 0.05418463945388794 Val Loss: 0.0597373821518638
Iteration: 176 Loss: 0.051368101437886555
Epoch: 7 Training Loss: 0.05136810143788655 Val Loss: 0.05896982776396202
Iteration: 176 Loss: 0.061881140867869066
Epoch: 8 Training Loss: 0.06188114086786906 Val Loss: 0.05859747032324472
Iteration: 176 Loss: 0.054642355442047124
Epoch: 9 Training Loss: 0.054642

## Measuring Model Uncertainty

In [12]:
def model_uncertainty(data, models, criterion, teacher_forcing_ratio):
    B = 10
    loss = 0
    
    encoder, decoder = models
    
    input_batch, target_batch = data
    
    batch_size = input_batch.shape[0]
    seq_length = input_batch.shape[1]
    num_features = input_batch.shape[2]

    encoder_hidden = encoder.initHidden()
    
    y_mc = torch.zeros(seq_length, batch_size, num_features).to(device)
    y_b = torch.zeros(B, seq_length, batch_size, num_features).to(device)
    eta_squared = torch.zeros(seq_length, batch_size, num_features).to(device)

    for b in range(B):
        
        for t in range(seq_length):
            input = input_batch[:, t, :].unsqueeze(0).float() # input of size 1 x batch_size x num_features
            _, encoder_hidden = encoder(input, encoder_hidden)

        decoder_hidden = encoder_hidden
        decoder_input = torch.ones_like(target_batch[:, 0, :]).unsqueeze(0).float()
        EOS = torch.zeros_like(target_batch[:, 0, :]).unsqueeze(0).float()

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        for t in range(seq_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            y_mc[t, :, :] += decoder_output.squeeze(0)
            y_b[b, t, :, :] = decoder_output
            
            target = target_batch[:, t, :].unsqueeze(0).float()
            loss += criterion(decoder_output, target)
            if use_teacher_forcing:
                decoder_input = target
            else:
                if torch.all(torch.eq(decoder_output, EOS)):
                    break
                decoder_input = decoder_output.detach()
    
    y_mc /= B
    
    for b in range(B):
        y_pred = y_b[b]
        eta_squared += torch.abs(y_pred - y_mc)
    
    eta = torch.sqrt(eta_squared / B)
    
    return loss.item() / (seq_length * B), eta, y_mc

            

In [22]:
def uncertainty_quantification(models, dataloaders, criterion, teacher_forcing_ratio):
    _, val_dataloader = dataloaders
    
    val_loss = 0
    etas = torch.zeros(len(val_dataloader), 120, 66)
    with torch.no_grad():
        # calculating eta
        for i, data in enumerate(val_dataloader, 0):
            loss, eta, y_mc = model_uncertainty(data, models, criterion, teacher_forcing_ratio)
            val_loss += loss
            etas[i, :, :] = torch.mean(eta, 1)
        
        val_loss /= len(val_dataloader)
        etas_from_features = torch.softmax(torch.max(torch.mean(eta, 0)), dim=0)
    return val_loss, etas_from_features

In [59]:
teacher_forcing_ratio = 0.0

losses, etas_from_features = uncertainty_quantification(models, dataloaders, criterion, teacher_forcing_ratio)
losses, etas_from_features = etas_from_featuers.cpu().numpy()

NameError: name 'criterion' is not defined

In [20]:
etas_from_features.shape

(44, 120, 66)

In [24]:
descending_most_uncertain = np.argsort(etas_from_features)[::-1]

joint_rotation_indices = [(i // 3, i % 3) for i in descending_most_uncertain]

In [25]:
joint_names = ['jL5S1','jL4L3','jL1T12','jT9T8','jT1C7','jC1Head',
          'jRightC7Shoulder','jRightShoulder','jRightElbow','jRightWrist',
          'jLeftC7Shoulder','jLeftShoulder','jLeftElbow','jLeftWrist',
          'jRightHip','jRightKnee','jRightAnkle','jRightBallFoot',
          'jLeftHip','jLeftKnee','jLeftAnkle','jLeftBallFoot']

#Euler  angle  extractions  of  Z (flexion/extension), X (abduction/adduction), Y (internal/external rotation)
angle_names = ['Flexion/extraction', 'Abduction/adduction', 'Internal/external rotation']

indices = np.arange(len(joints))+1

joint_indices = dict(zip(indices, joint_names))

angle_indices = dict(zip([1, 2, 3], angle_names))

joint_rotation_indices = [(joint_indices[i+1], angle_indices[j+1]) for i, j in joint_rotation_indices]

joint_rotation_indices

[('jLeftElbow', 'Abduction/adduction'),
 ('jLeftElbow', 'Internal/external rotation'),
 ('jRightElbow', 'Abduction/adduction'),
 ('jRightElbow', 'Internal/external rotation'),
 ('jLeftHip', 'Abduction/adduction'),
 ('jRightElbow', 'Flexion/extraction'),
 ('jLeftElbow', 'Flexion/extraction'),
 ('jRightWrist', 'Internal/external rotation'),
 ('jLeftKnee', 'Abduction/adduction'),
 ('jRightShoulder', 'Internal/external rotation'),
 ('jRightWrist', 'Flexion/extraction'),
 ('jLeftShoulder', 'Internal/external rotation'),
 ('jLeftHip', 'Internal/external rotation'),
 ('jRightHip', 'Internal/external rotation'),
 ('jRightShoulder', 'Abduction/adduction'),
 ('jC1Head', 'Abduction/adduction'),
 ('jLeftShoulder', 'Abduction/adduction'),
 ('jLeftShoulder', 'Flexion/extraction'),
 ('jRightShoulder', 'Flexion/extraction'),
 ('jLeftKnee', 'Internal/external rotation'),
 ('jLeftWrist', 'Internal/external rotation'),
 ('jT1C7', 'Internal/external rotation'),
 ('jRightWrist', 'Abduction/adduction'),
 ('

# Training with Model Uncertainty

In [23]:
def get_encoder(num_features, hidden_size=64, lr=0.001, bs=batch_size):
    encoder = EncoderRNN(num_features, hidden_size, bs).to(device)
    return encoder, optim.Adam(encoder.parameters(), lr=lr)

def get_decoder(num_features, hidden_size=64, lr=0.001, bs=batch_size):
    decoder = DecoderRNN(num_features, hidden_size, num_features, bs).to(device)
    return decoder, optim.Adam(decoder.parameters(), lr=lr)

In [24]:
def weighted_l1_loss(input, target, weights, reduction='weighted'):
    if target.requires_grad:
        ret = torch.abs(input - target)
        if reduction != 'none':
            ret = torch.dot(ret, weights) if reduction == 'weighted' else torch.mean(ret)
    else:
        expanded_input, expanded_target = torch.broadcast_tensors(input, target)
        ret = torch.abs(expanded_input - expanded_target)
        if reduction != 'none':
            ret = torch.dot(ret, weights) if reduction == 'weighted' else torch.mean(ret)
    return ret

class Weighted_L1Loss(torch.nn.modules.loss._Loss):
    __constants__ = ['reduction']

    def __init__(self, reduction='weighted'):
        super(Weighted_L1Loss, self).__init__(reduction)

    def forward(self, input, target, weights):
        return weighted_l1_loss(input, target, weights, reduction=self.reduction)

In [25]:
def train(data, models, opts, criterion, teacher_forcing_ratio, weights):
    loss = 0
    
    encoder, decoder = models
    
    if opts is not None:
        encoder_opt, decoder_opt = opts 
    
    input_batch, target_batch = data
    
    seq_length = input_batch.shape[1]

    encoder_hidden = encoder.initHidden()

    for t in range(seq_length):
        input = input_batch[:, t, :].unsqueeze(0).float()
        _, encoder_hidden = encoder(input, encoder_hidden)

    decoder_hidden = encoder_hidden
    decoder_input = torch.ones_like(target_batch[:, 0, :]).unsqueeze(0).float()
    EOS = torch.zeros_like(target_batch[:, 0, :]).unsqueeze(0).float()

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    for t in range(seq_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        target = target_batch[:, t, :].unsqueeze(0).float()
        if weights is None:
            loss += criterion(decoder_output, target)
        else:
            loss += criterion(decoder_output, target, weights)
            
        if torch.all(torch.eq(decoder_output, EOS)):
            break
        
        if use_teacher_forcing:
            decoder_input = target
        else:
            decoder_input = decoder_output.detach()

    if opts is not None:
        loss.backward()
        
        encoder_opt.step()
        encoder_opt.zero_grad()

        decoder_opt.step()
        decoder_opt.zero_grad()
    
    return loss.item() / seq_length

In [26]:
def fit(models, opts, epochs, dataloaders, criteria, teacher_forcing_ratio, weights):
    
    train_dataloader, val_dataloader = dataloaders
    Weighted_L1Loss, L1Loss = criteria
    
    for epoch in range(epochs):

        for index, data in enumerate(train_dataloader, 0):
            loss = train(data, models, 
                         opts, Weighted_L1Loss,
                         teacher_forcing_ratio, weights) 
            print("Iteration: " + str(index + 1), "Loss: " + str(loss), end="\r")
        with torch.no_grad():
            # calculating loss for validation
            losses = [train(data, models, None, L1Loss, teacher_forcing_ratio, None) 
                      for _, data in enumerate(val_dataloader, 0)]
        val_loss = np.sum(losses) / len(losses)

        _, eta_from_features = uncertainty_quantification(models, dataloaders, L1Loss, teacher_forcing_ratio)    
        weights = eta_from_features
        print()
        print("Epoch: " + str(epoch + 1), "Training Loss: " + str(loss), "Val Loss: " + str(val_loss))

In [27]:
epochs = 10

encoder, encoder_opt = get_encoder(66)
decoder, decoder_opt = get_decoder(66)
teacher_forcing_ratio = 0.0

models = (encoder, decoder)
opts = (encoder_opt, decoder_opt)
weights = torch.ones(66) / 66.
dataloaders = (train_dataloader, val_dataloader)

criteria = (Weighted_L1Loss(), nn.L1Loss())
loss = 0

fit(models, opts, epochs, dataloaders, criteria, teacher_forcing_ratio, weights)

Iteration: 176 Loss: 0.08187534014383951
Epoch: 1 Training Loss: 0.08187534014383951 Val Loss: 0.08233602841695149
Iteration: 176 Loss: 0.06941592693328857
Epoch: 2 Training Loss: 0.06941592693328857 Val Loss: 0.06888229846954347
Iteration: 176 Loss: 0.074510796864827484
Epoch: 3 Training Loss: 0.07451079686482748 Val Loss: 0.06471657653649648
Iteration: 176 Loss: 0.055969893932342534
Epoch: 4 Training Loss: 0.05596989393234253 Val Loss: 0.062103370644829486
Iteration: 176 Loss: 0.065044792493184414
Epoch: 5 Training Loss: 0.06504479249318441 Val Loss: 0.06003641016555555
Iteration: 176 Loss: 0.062653764088948574
Epoch: 6 Training Loss: 0.06265376408894857 Val Loss: 0.05916433641404817
Iteration: 176 Loss: 0.061070084571838384
Epoch: 7 Training Loss: 0.06107008457183838 Val Loss: 0.058655395652308616
Iteration: 176 Loss: 0.053784346580505374
Epoch: 8 Training Loss: 0.05378434658050537 Val Loss: 0.05804782816858003
Iteration: 176 Loss: 0.056990563869476325
Epoch: 9 Training Loss: 0.0569